# 3. Feature Engineering and data preparation
* Now we have performed EDA and done preliminary cleaning of the data we can now start preparing the data ready for usage by our chosen algorithm
    * Decided upon LGB as 
        * 1. it has demonstrated good performance on this problem due to work by Wenlong et al. on previous competition
        * 2. intrinsically handles problems associated with the multiple instances as DT's don't require categorical values to be one hot encoded (this actually decreases performance)
            * i. means we only need to use one model instead of multiple; much faster (having a separate RNN for each instance would be infeasible given the time constraints I am working under)
            * ii. don't have issue of one hot encoding 3500 unique categorical values causing a sparse solution space and hence making a singular RNN impractical as it would either have to treat all values as one continuous sequence or use ordinal encoded meter_id where it will impose a non existant ordinal relationship or onehot encode meter_id which would add 3500 extra features and make solution space too sparse
            
* Going to restructure data for the LGB and create some features which may be useful in aiding model to pick up the underlying relationships

# Imports


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime

# pd.display optioons

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# loading data

In [3]:
df_energy = pd.read_pickle("../Data/Preprocessed_Data/consumption_clustered.pkl")
df_energy

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-08,2017-12-09,2017-12-10,2017-12-11,2017-12-12,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7410,5.48250,7.224,4.951,4.19650,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,5.1075,4.818,3.931,4.2170,4.503,4.8160,5.129,5.395,5
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.4690,24.09950,24.730,24.597,22.32650,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,14.6315,14.936,16.174,20.3960,24.618,19.8925,15.167,11.751,3
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0265,16.22475,12.423,18.379,15.85025,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,7.3520,7.384,14.425,16.0650,17.705,13.3355,8.966,4.633,1
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.3150,21.49350,19.672,21.023,20.35450,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,11.7255,10.974,19.646,21.8195,23.993,19.9170,15.841,14.452,3
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.3880,41.99750,48.607,54.760,48.73150,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,21.9445,8.351,9.957,17.9140,25.871,36.0725,46.274,16.901,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,2.3040,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,9.9760,10.30550,10.635,10.650,10.39950,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,9.2235,9.216,9.336,9.6840,10.032,9.8945,9.757,9.480,1
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,10.3010,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,5.8620,6.81600,7.770,5.091,5.60000,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,3.8230,4.818,3.357,8.1315,12.906,8.0140,3.122,3.401,5
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,6.8355,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,6.449,...,7.2410,6.46800,5.695,8.143,7.56800,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,5.9980,6.077,7.761,6.6080,5.455,5.5670,5.679,8.148,5
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,24.6880,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,28.7260,29.28500,29.844,31.899,30.96900,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,20.1925,20.406,23.668,27.7900,31.912,29.1125,26.313,24.201,0


In [5]:
df_addInfo = pd.read_pickle("../Data/Preprocessed_Data/addInfo_knn.pkl")
df_addInfo

,meter_id,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0x02b7418b821b2edeeb2baa23ba1580566b4b0e43,3.0,0.0,0.0,1.0,0.0
1,0xaf32b2b497e9eb9c3f3de907aedc6a7914a11f41,2.0,0.0,0.0,0.0,1.0
2,0xdd45ab28a7a40eadc04607d6da42e5ad923b1339,3.0,0.0,0.0,1.0,0.0
3,0x2b538a5ebfc3780aef25227267063cb6410d943b,4.0,1.0,0.0,0.0,0.0
4,0xee032ed498c9e99a4c65e0f39896fb30e513c6b7,3.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
3243,0xdf6a54b92985398af03423426e744c6d40ee8c69,4.0,0.0,0.0,1.0,0.0
3244,0x6b6ddc9a73405caab50c8d5575b72288aaa7e844,3.0,1.0,0.0,0.0,0.0
3245,0xd67e6b5f31d5286a2d4eec159b972e7f2197a67b,3.0,0.0,0.0,1.0,0.0
3246,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,3.0,0.0,0.0,1.0,0.0


In [6]:
# joining based on meter_id to assign labels to the original dataset (with all meter readings)
df_energy_addInfo = pd.merge(df_energy, df_addInfo, on="meter_id",how="inner")
df_energy_addInfo

,meter_id,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,...,2017-12-13,2017-12-14,2017-12-15,2017-12-16,2017-12-17,2017-12-18,2017-12-19,2017-12-20,2017-12-21,2017-12-22,2017-12-23,2017-12-24,2017-12-25,2017-12-26,2017-12-27,2017-12-28,2017-12-29,2017-12-30,2017-12-31,labels,num_bedrooms,dwelling_type_detached_house,dwelling_type_flat,dwelling_type_semi_detached_house,dwelling_type_terraced_house
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.4420,3.509,4.6750,6.131,4.962,3.336,4.318,3.561,4.279,4.030,5.397,5.1075,4.818,3.931,4.2170,4.503,4.8160,5.129,5.395,5,2.0,0.0,0.0,0.0,1.0
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.0560,22.546,22.3330,23.353,22.774,20.229,21.218,15.419,13.475,13.101,14.327,14.6315,14.936,16.174,20.3960,24.618,19.8925,15.167,11.751,3,4.0,1.0,0.0,0.0,0.0
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.3215,13.821,14.2175,16.712,12.364,16.102,18.601,10.385,10.917,10.201,7.320,7.3520,7.384,14.425,16.0650,17.705,13.3355,8.966,4.633,1,3.0,0.0,0.0,1.0,0.0
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.6860,23.435,18.7330,22.249,20.755,19.730,21.135,16.526,12.677,14.833,12.477,11.7255,10.974,19.646,21.8195,23.993,19.9170,15.841,14.452,3,3.0,0.0,0.0,1.0,0.0
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.7030,35.045,38.6640,36.790,29.009,42.716,29.052,25.444,28.234,39.440,35.538,21.9445,8.351,9.957,17.9140,25.871,36.0725,46.274,16.901,0,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,0x7dd7a7b8ee1bec7c44b24f738c752482f6161065,2.317,2.301,2.352,2.516,2.229,2.354,2.397,2.397,2.269,2.3040,2.339,2.212,2.456,2.229,2.503,2.207,2.469,2.266,2.321,2.262,2.286,2.384,2.319,2.289,...,10.1490,10.112,10.6320,10.566,10.732,10.615,10.386,9.936,9.625,9.390,9.231,9.2235,9.216,9.336,9.6840,10.032,9.8945,9.757,9.480,1,3.0,0.0,0.0,1.0,0.0
3244,0xfdaf9f857621ec06f2cf801f42a020a322835090,14.437,16.274,7.031,17.018,17.603,15.005,8.987,8.490,10.136,10.3010,10.466,8.177,11.929,8.261,12.369,14.352,14.058,10.472,17.396,9.879,21.971,12.708,15.327,8.309,...,6.1090,5.056,3.9720,3.458,7.793,9.837,8.697,5.079,6.031,4.141,2.828,3.8230,4.818,3.357,8.1315,12.906,8.0140,3.122,3.401,5,2.0,0.0,0.0,0.0,1.0
3245,0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18,7.824,7.517,5.398,6.788,7.360,6.898,7.321,8.042,8.207,6.8355,5.464,7.545,6.432,6.001,5.558,6.391,6.066,6.443,6.675,7.363,7.269,7.183,7.102,6.449,...,6.9930,6.451,7.6290,6.880,5.960,6.231,5.748,5.259,5.258,6.767,5.919,5.9980,6.077,7.761,6.6080,5.455,5.5670,5.679,8.148,5,2.0,0.0,0.0,0.0,1.0
3246,0x47218b46abb2fcaade487a211911406dc6e13730,23.965,28.689,27.664,29.229,29.548,27.909,26.923,21.277,23.452,24.6880,25.924,26.834,30.346,30.367,26.135,26.842,25.588,30.267,28.921,27.530,28.424,28.961,29.346,28.640,...,30.0390,30.470,30.0440,28.617,31.578,31.481,32.123,29.524,22.586,20.747,19.979,20.1925,20.406,23.668,27.7900,31.912,29.1125,26.313,24.201,0,3.0,1.0,0.0,0.0,0.0


# Training set 
## Restructuring for the LGB model
* Training set is the data we have available

* Converting to a supervised learning problem
    * [X,y]
        * X = ["meter_id","date","label"]
        * y = ["meter_reading"]

In [79]:
#structure for energy training set
df_energy_train = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced","meter_reading"])

#iteratively populating the training set
date = pd.date_range(datetime(2017, 1, 1), datetime(2017, 12, 31))#outside of loop as doesn't need to be recaculated

#for each row in the dataframe
for row_id in tqdm(range(len(df_energy_addInfo))):
    
    #getting relevant values for this row
    row = df_energy_addInfo.iloc[row_id]
    
    meter_id = row.meter_id
    energy_cluster = row.labels
    num_bedrooms = row.num_bedrooms
    detached = row.dwelling_type_detached_house
    flat = row.dwelling_type_flat
    semi_detached = row.dwelling_type_semi_detached_house
    terraced = row.dwelling_type_terraced_house
    meter_reading = row.iloc[1:366].values.astype(float)
    
    #formatting this row into dataframe with timeseries patterns over rows instead of columns
    df_meter = pd.DataFrame(columns=["meter_id","date","energy_cluster","num_bedrooms","detached","flat","semi_detached","terraced","meter_reading"])
    
    df_meter["date"]=date #needs to be first as it determines length of columns
    df_meter["meter_id"]=meter_id
    df_meter["energy_cluster"]=energy_cluster
    df_meter["num_bedrooms"]=num_bedrooms
    df_meter["detached"]=detached
    df_meter["flat"]=flat
    df_meter["semi_detached"]=semi_detached
    df_meter["terraced"]=terraced
    df_meter["meter_reading"] = meter_reading
    
    #dataframe restructuring this row into columns 
    df_energy_train = pd.concat([df_energy_train, df_meter]).reset_index(drop=True)
    
df_energy_train

  0%|          | 0/3248 [00:00<?, ?it/s]

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-01,5,2.0,0.0,0.0,0.0,1.0,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-02,5,2.0,0.0,0.0,0.0,1.0,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-03,5,2.0,0.0,0.0,0.0,1.0,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-04,5,2.0,0.0,0.0,0.0,1.0,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-01-05,5,2.0,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,0,3.0,1.0,0.0,0.0,0.0,18.7585
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,0,3.0,1.0,0.0,0.0,0.0,21.0110
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,0,3.0,1.0,0.0,0.0,0.0,19.2040
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,0,3.0,1.0,0.0,0.0,0.0,17.3970


In [80]:
# number of nans
(df_energy_train.meter_reading.isna()).sum()

545221

In [83]:
# removing rows containing nans as the model won't be able to use them
df_energy_train = df_energy_train.dropna(axis=0)
df_energy_train

,meter_id,date,energy_cluster,num_bedrooms,detached,flat,semi_detached,terraced,meter_reading
336,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-03,5,2.0,0.0,0.0,0.0,1.0,3.5170
337,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-04,5,2.0,0.0,0.0,0.0,1.0,2.9330
338,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-05,5,2.0,0.0,0.0,0.0,1.0,3.0850
339,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-06,5,2.0,0.0,0.0,0.0,1.0,3.2370
340,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,2017-12-07,5,2.0,0.0,0.0,0.0,1.0,4.4710
...,...,...,...,...,...,...,...,...,...
1185515,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-27,0,3.0,1.0,0.0,0.0,0.0,18.7585
1185516,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-28,0,3.0,1.0,0.0,0.0,0.0,21.0110
1185517,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-29,0,3.0,1.0,0.0,0.0,0.0,19.2040
1185518,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,2017-12-30,0,3.0,1.0,0.0,0.0,0.0,17.3970


In [84]:
# number of nans
(df_energy_train.meter_reading.isna()).sum()

0

## Feature engineering 
#### Dummy variables for the month and day 